In [3]:
import pandas as pd
from zone_recovery.zone import *

In [3]:
test = Zone(1,2,3,4)

In [34]:
df  = pd.read_csv('processed_data/BNBUSTD_2019_Processed.csv')

In [52]:
df[0]

KeyError: 0

In [55]:
for i,r in df[5:10].iterrows():
    print(r.Price + 5)

11.1162
11.117999999999999
11.1196
11.1172
11.1065


In [6]:
for i,row in df[0:10].iterrows():
    print(i)

0
1
2
3
4
5
6
7
8
9


In [4]:
import numpy as numpy
import pandas as pd

class Zone:
    def __init__(self, entry_price, zone_width, distance):
        "entry_price"
        self.up = entry_price
        self.down = entry_price - zone_width
        self.above_target = entry_price + distance
        self.below_target = entry_price - zone_width - distance

    def calculateProfit(self, t, df, money):
        state = 'Sell'
        first_money = money
        buy_cost = 0
        sell_cost = 0

        buy_unit = 0
        sell_unit = 0
        alpha = 1

        temp_money = 0

        for i, row in df.iterrows():

            if row.Price > buy_cost and state == 'Buy':
                buy_cost += 50 * alpha
                alpha *= 2
                money -= buy_cost
                buy_unit += buy_cost / row.Price
                state = 'Sell'
                print('Buy', buy_cost)

            if row.Price > sell_cost and state == 'Sell':
                sell_cost += 50 * alpha
                temp_money += sell_cost

                alpha *= 2
                money -= sell_cost
                sell_unit += sell_cost / row.Price
                state = 'Buy'
                print('Sell', sell_cost)

            if row.Price > self.above_target and state == 'Sell':
                buy_cost = 0
                money += buy_unit * row.Price

                state = 'Buy'
                print('Close Buy', buy_unit * row.Price)
                buy_unit = 0
                alpha = 1

            if row.Price < self.below_target and state == 'Buy':
                sell_cost = 0
                debt = sell_unit * row.Price
                money += temp_money - debt
                sell_unit = 0
                state = 'Sell'
                print('Close Sell', temp_money - debt)
                alpha = 1

        return money - first_money

    def monitorZone(self):
        return self.up, self.down, self.above_target, self.below_target


In [40]:
first_price = df.iloc[0].Price

In [41]:
first_price

6.1222

In [14]:
z = Zone(1,2,3)

In [15]:
z.monitorZone()

(1, -1, 4, -4)

In [36]:
def simulation(df,limit=4,expected_profit = 4,alpha=2,money=1000,zone_width = 1, distance = 4):


    state = 'w8 4 Sell'
    action = 'Open Buy'
    first_money = money
    temp_money = 0
    
    buy_cost =  1 
    sell_cost = 0
    
    sell_unit = 0
    
    first_price = df.iloc[0].Price
    prev_price = first_price
    buy_unit = buy_cost / first_price

    money -= buy_cost
    print(action,first_price,money,1,0)

    zone = Zone(first_price,first_price * zone_width / 100, first_price * distance / 100)

    buy_hold = 1
    sell_hold = 0
    
    for i,row in df.iterrows():
        
        current_price = row.Price
        action, state = makeDecision(prev_price, current_price,zone,state)
        
        if buy_hold == 0 and sell_hold == 0:
            zone = Zone(current_price,current_price * zone_width / 100, current_price* distance / 100)
            state = 'w8 4 Sell'
            action = 'Open Buy'

        if buy_hold + sell_hold > limit+10:
            action = 'Cut Loss'

        if action = 'Cut Loss':

            buy_cost = 0
            money += buy_unit * row.Price
            buy_unit = 0
          

            sell_cost = 0
            debt = sell_unit * row.Price
            money += temp_money - debt
            sell_unit = 0
            buy_hold,sell_hold = 0,0
            alpha = 1
            temp_money = 0
            action = 'Cut Loss'
    
        
        if action == 'Open Buy':
            buy_cost += 1 * alpha
            buy_hold+= alpha
            alpha *= 2
            money -= buy_cost
            buy_unit += buy_cost / row.Price
            
            # state = 'Sell'
            # print('Open Buy', buy_cost, row.Price,money)
            

        if action == 'Open Sell':
            sell_cost += 1 * alpha
            sell_hold+=  alpha
            temp_money += sell_cost
            alpha *= 2
            money -= sell_cost
            sell_unit += sell_cost / row.Price
         
            # state = 'Buy'
            # print('Open Sell', sell_cost,money)


        if action == 'Close Buy':
            

            buy_cost = 0
            money += buy_unit * row.Price

            # state = 'Buy'
            # print('Close Buy', buy_unit * row.Price,money)
            buy_hold = 0
            buy_unit = 0
            alpha = 1
            

        if action == 'Close Sell':
            
            
            sell_cost = 0
            debt = sell_unit * row.Price
            print(temp_money,debt)
            money += 2*temp_money - debt
            sell_unit = 0
            sell_hold = 0
            temp_money = 0
            # state = 'Sell'
            # print('Close Sell', temp_money - debt,money)
            alpha = 1

        if sell_unit == 0 and buy_unit == 0:
            zone = Zone(current_price,current_price* zone_width / 100, current_price* distance / 100)



        if(action != 'Steady'):
            print(action,current_price,money,buy_hold,sell_hold)

        prev_price = current_price
    print(money-first_money)





SyntaxError: invalid syntax (<ipython-input-36-d1fb169b5607>, line 39)

In [25]:
def makeDecision(prev_price, current_price, zone: Zone , state : str):

    if (prev_price > zone.up and current_price < zone.up or current_price == zone.up) and state == 'w8 4 Buy':
        return 'Open Buy', 'w8 4 Sell'

    if (prev_price < zone.down and current_price > zone.down or current_price == zone.down) and state == 'w8 4 Sell':
        return 'Open Sell', 'w8 4 Buy'

    if (prev_price < zone.above_target and current_price > zone.above_target or current_price == zone.above_target) and state == 'w8 4 Sell':
        return 'Close Buy', 'w8 4 Buy'

    if (prev_price > zone.below_target and current_price < zone.below_target or current_price == zone.below_target) and state == 'w8 4 Buy':
        return 'Close Sell', 'w8 4 Buy'

    return 'Steady', state



In [ ]:
def takeAction(action,money,buy_cost,sell_cost,buy_unit,sell_unit,temp_money)

In [135]:
def simulation(df, entry_amount, limit=99, expected_profit=4, multiplier=2, money=100, zone_width=1, distance=4):

    buy_unit, sell_unit, buy_hold, sell_hold = 0, 0, 0, 0
    alpha = 1

    log = []
    for i, current in df.iterrows():

        if buy_hold == 0 and sell_hold == 0:
            zone = Zone(current.Price, current.Price *
                        zone_width / 100, current.Price * distance / 100)
            state, action = 'w8 4 Sell', 'Open Buy'

        else:
            action, state = makeDecision(
                prev_price, current.Price, zone, state)

        if entry_amount * alpha > money:
            action = 'Cut Loss'

        if action == 'Open Buy':
            buy_hold += entry_amount*alpha
            money -= entry_amount*alpha
            buy_unit += entry_amount*alpha / current.Price
            alpha *= multiplier

        if action == 'Open Sell':
            sell_hold += entry_amount*alpha
            money -= entry_amount*alpha
            sell_unit += entry_amount*alpha / current.Price
            alpha *= multiplier

        if action == 'Cut Loss' or action == 'Close Buy' or action == 'Close Sell':
            money += buy_unit * current.Price + 2*sell_hold - sell_unit * current.Price
            buy_unit = 0
            sell_unit = 0
            buy_hold, sell_hold = 0, 0
            alpha = 1
            state = 'w8 4 Sell'

        if(action != 'Steady'):
            print(action, current.Price, money, buy_hold, sell_hold)

        log.append((action, money, buy_hold, sell_hold))
        prev_price = current.Price
    return log

In [130]:
x[0:10]

[('Open Buy', 6.1222, 88, 12, 0),
 ('Steady', 6.1215, 88, 12, 0),
 ('Steady', 6.122999999999999, 88, 12, 0),
 ('Steady', 6.1161, 88, 12, 0),
 ('Steady', 6.12, 88, 12, 0),
 ('Steady', 6.1162, 88, 12, 0),
 ('Steady', 6.117999999999999, 88, 12, 0),
 ('Steady', 6.1196, 88, 12, 0),
 ('Steady', 6.1172, 88, 12, 0),
 ('Steady', 6.1065, 88, 12, 0)]

In [136]:
x = simulation(df,entry_amount = 1,money = 10)

Open Buy 6.1222 9 1 0
Open Sell 6.068 7 1 2
Close Sell 5.8126 10.03360924473684 0 0
Open Buy 5.8260000000000005 9.03360924473684 1 0
Close Buy 6.0608 10.073911338797947 0 0
Open Buy 6.0625 9.073911338797947 1 0
Open Sell 6.002999999999999 7.073911338797947 1 2
Close Sell 5.7535 10.106067370747608 0 0
Open Buy 5.7559 9.106067370747608 1 0
Close Buy 5.9862 10.146078489773304 0 0
Open Buy 5.977 9.146078489773304 1 0
Open Sell 5.9211 7.146078489773304 1 2
Open Buy 5.9751 3.146078489773304 5 2
Cut Loss 5.9519 10.115944465474714 0 0
Open Buy 5.9562 9.115944465474714 1 0
Close Buy 6.2 10.15687660341501 0 0
Open Buy 6.2906 9.15687660341501 1 0
Open Sell 6.2292 7.156876603415011 1 2
Open Buy 6.2875 3.156876603415011 5 2
Cut Loss 6.2811 10.134631387049952 0 0
Open Buy 6.2658 9.134631387049952 1 0
Open Sell 6.2067 7.134631387049952 1 2
Open Buy 6.2657 3.134631387049952 5 2
Cut Loss 6.257000000000001 10.111464598227247 0 0
Open Buy 6.2529 9.111464598227247 1 0
Open Sell 6.1942 7.111464598227247 1 

In [110]:
def simulation0(df, limit=0, expected_profit=4, alpha=1, money=10, zone_width=1, distance=4):

    money = limit

    temp_money, buy_cost, sell_cost = 0, 0, 0

    buy_unit, sell_unit, buy_hold, sell_hold = 0, 0, 0, 0

    profit = 0

    for i, row in df.iterrows():

        current_price = row.Price

        if buy_hold == 0 and sell_hold == 0:
            zone = Zone(current_price, current_price *
                        zone_width / 100, current_price * distance / 100)
            state, action = 'w8 4 Sell', 'Open Buy'

        else:
            action, state = makeDecision(
                prev_price, current_price, zone, state)

        if buy_hold + sell_hold > limit:
            action, state = 'Cut Loss', 'w8 4 Sell'

        if action == 'Cut Loss' or action == 'Close Buy' or action == 'Close Sell':

            buy_cost = 0
            money += buy_unit * row.Price
            buy_unit = 0

            sell_cost = 0
            debt = sell_unit * row.Price
            money += 2*temp_money - debt
            sell_unit = 0
            buy_hold, sell_hold = 0, 0
            alpha = 1
            temp_money = 0

        if action == 'Open Buy':
            if buy_hold + sell_hold + alpha > limit:
                action = 'Cut Loss'
                pass
            buy_cost += 1 * alpha
            buy_hold += alpha
            alpha *= 2
            money -= buy_cost
            buy_unit += buy_cost / row.Price

            # state = 'Sell'
            # print('Open Buy', buy_cost, row.Price,money)

        if action == 'Open Sell':
            if buy_hold + sell_hold + alpha > limit:
                action = 'Cut Loss'
                pass

            sell_cost += 1 * alpha
            sell_hold += alpha
            temp_money += sell_cost
            alpha *= 2
            money -= sell_cost
            sell_unit += sell_cost / row.Price

        if action == 'Cut Loss' or action == 'Close Buy' or action == 'Close Sell':

            buy_cost = 0
            money += buy_unit * row.Price
            buy_unit = 0

            sell_cost = 0
            debt = sell_unit * row.Price
            money += 2*temp_money - debt
            sell_unit = 0
            buy_hold, sell_hold = 0, 0
            alpha = 1
            temp_money = 0

            # state = 'Buy'
            # print('Open Sell', sell_cost,money)

        # if sell_unit == 0 and buy_unit == 0:
        #     zone = Zone(current_price,current_price* zone_width / 100, current_price* distance / 100)

        if(action != 'Steady'):
            print(action, current_price, money, buy_hold, sell_hold)

        prev_price = current_price
    return money


In [27]:
test_zone.monitorZone()

(10, 5, 15, 0)

In [7]:
makeDecision(-2,-1,test_zone,'w8 4 Buy')

NameError: name 'test_zone' is not defined

In [113]:
df4 = df[ df['Minute']% == 0]

In [65]:
simulation_minimize(df,limit = 50)

Open Buy 6.1222 1 1 0
Open Sell 6.068 2 1 2
Close Sell 5.8126 0 0 0
Open Buy 5.8260000000000005 1 1 0
Close Buy 6.0608 0 0 0
Open Buy 6.0625 1 1 0
Open Sell 6.002999999999999 2 1 2
Close Sell 5.7535 0 0 0
Open Buy 5.7559 1 1 0
Close Buy 5.9862 0 0 0
Open Buy 5.977 1 1 0
Open Sell 5.9211 2 1 2
Open Buy 5.9751 3 5 2
Close Buy 6.2906 0 0 0
Open Buy 6.2603 1 1 0
Open Sell 6.1999 2 1 2
Open Buy 6.2534 3 5 2
Open Sell 6.2075 4 5 10
Open Buy 6.2437 5 21 10
Cut Loss 6.1979999999999995 0 0 0
Open Buy 6.1979999999999995 1 1 0
Open Sell 6.1413 2 1 2
Open Buy 6.1953 3 5 2
Close Buy 6.447 0 0 0
Open Buy 6.4424 1 1 0
Close Buy 6.7089 0 0 0
Open Buy 6.7023 1 1 0
Open Sell 6.641 2 1 2
Open Buy 6.7016 3 5 2
Open Sell 6.6375 4 5 10
Close Sell 6.349 0 0 0
Open Buy 6.3559 1 1 0
Open Sell 6.2953 2 1 2
Open Buy 6.3512 3 5 2
Open Sell 6.2933 4 5 10
Open Buy 6.3541 5 21 10
Cut Loss 6.2951 0 0 0
Open Buy 6.2908 1 1 0
Close Buy 6.5491 0 0 0
Open Buy 6.5542 1 1 0
Open Sell 6.5099 2 1 2
Open Buy 6.5456 3 5 2
Open

2.177353293415701